In [23]:
# IMPORT AND DISPLAY SETTINGS


import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time
import mlflow
import mlflow.sklearn
import mlflow.lightgbm
import mlflow.xgboost

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, average_precision_score, ConfusionMatrixDisplay

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier



# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format



In [24]:
user_data = pd.read_csv(r"processed-data\loan-processed-data.csv")

In [25]:
# user_data.isna().sum().reset_index()
# No null values

In [26]:
user_data['Default'] = user_data['Default'].astype(int)
# Sanitize column names globally
user_data.columns = [col.strip().replace(" ", "_") for col in user_data.columns]

user_data.head()

,Client_Income_Type_Govt_Job,Client_Income_Type_Others,Client_Income_Type_Retired,Client_Income_Type_Service,Client_Income_Type_Unemployed,Client_Education_Graduation_dropout,Client_Education_Junior_secondary,Client_Education_Post_Grad,Client_Education_Secondary,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W,Client_Gender_Male,Loan_Contract_Type_RL,Client_Housing_Type_Home,Client_Housing_Type_Municipal,Client_Housing_Type_Office,Client_Housing_Type_Rental,Client_Housing_Type_Shared,Client_Permanent_Match_Tag_Yes,Client_Contact_Work_Tag_Yes,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,Population_Region_Relative,Age_Days,Employed_Days,Registration_Days,ID_Days,Mobile_Tag,Homephone_Tag,Workphone_Working,Client_Family_Members,Cleint_City_Rating,Application_Process_Day,Application_Process_Hour,Score_Source_2,Score_Source_3,Phone_Change,Credit_Bureau,Default,Occupation_Risk_Level,Org_Type_Binned
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12142509.00,6750.00,0.00,0.00,1.00,0.00,0.00,61190.55,3416.85,0.00,0.03,13957.00,1062.00,6123.00,383.00,1.00,0.00,0.00,2.00,2.00,6.00,17.00,0.48,0.55,63.00,1.00,0,2.00,3.00
1,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12138936.00,20250.00,1.00,0.00,1.00,1.00,0.00,15282.00,1826.55,0.00,0.01,14162.00,4129.00,7833.00,21.00,1.00,0.00,1.00,2.00,2.00,3.00,10.00,0.22,0.55,755.00,1.00,0,2.00,2.00
2,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,12181264.00,18000.00,0.00,0.00,1.00,0.00,1.00,59527.35,2788.20,0.00,0.02,16790.00,5102.00,4493.00,331.00,1.00,0.00,0.00,2.00,2.00,4.00,12.00,0.55,0.33,277.00,0.00,0,2.00,3.00
3,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12188929.00,15750.00,0.00,0.00,1.00,1.00,0.00,53870.40,2295.45,0.00,0.01,23195.00,12019.50,4493.00,775.00,1.00,0.00,0.00,2.00,2.00,2.00,15.00,0.14,0.63,1700.00,3.00,0,2.00,1.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,12133385.00,33750.00,1.00,0.00,1.00,0.00,2.00,133988.40,3547.35,0.00,0.02,11366.00,2977.00,5516.00,4043.00,1.00,0.00,0.00,4.00,2.00,3.00,12.00,0.30,0.36,674.00,1.00,0,2.00,3.00


In [27]:

# Set MLflow experiment
mlflow.set_experiment("Loan Default Model Comparison V4")

# Columns to scale
columns_to_scale = [
    'Client_Income', 'Credit_Amount', 'Loan_Annuity',
    'Population_Region_Relative', 'Age_Days', 'Employed_Days',
    'Registration_Days', 'ID_Days', 'Score_Source_2',
    'Score_Source_3', 'Phone_Change', 'Credit_Bureau',
    'Client_Family_Members'
]

# Split and scale
def stratified_split_and_scale(df, target_col='Default', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=test_size, random_state=random_state
    )

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
    X_test_scaled[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

    return X_train_scaled, X_test_scaled, X_train, X_test, y_train, y_test, scaler

# Resampling logic
def apply_resampling(X, y, method='none'):
    print(f"\n🔁 {method.upper()} - Before: {Counter(y)}")
    if method == 'undersample':
        sampler = RandomUnderSampler(random_state=42)
    elif method == 'oversample':
        sampler = RandomOverSampler(random_state=42)
    elif method == 'smote':
        sampler = SMOTE(random_state=42)
    elif method == 'smote_tomek':
        sampler = SMOTETomek(random_state=42)
    else:
        return X, y

    X_res, y_res = sampler.fit_resample(X, y)
    print(f"✅ {method.upper()} - After: {Counter(y_res)}")
    return X_res, y_res

# Evaluation logic
def evaluate_with_mlflow(model, X_test, y_test, model_name, sampling_method, X_train, y_train, train_time, custom_threshold):
    # custom_threshold = 0.3
    with mlflow.start_run(run_name=f"{model_name} with {sampling_method} with threshold {custom_threshold}" ):

        mlflow.set_tag("dataset", "loan_default_v1")
        # Log parameters
        mlflow.log_param("Model", model_name)
        mlflow.log_param("Sampling", sampling_method)
        mlflow.log_param("custom_threshold", custom_threshold)
        mlflow.log_param("Dataset", "loan_default_v1" )

        
# Check if model supports probability output
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]
            y_pred = (y_proba >= custom_threshold).astype(int)

            # Log PR AUC
            pr_auc = average_precision_score(y_test, y_proba)
            mlflow.log_metric("PR_AUC", pr_auc)
        else:
            print(f"{model_name} does not support predict_proba. Skipping threshold logic and PR AUC.")
            y_pred = model.predict(X_test)
            y_proba = None  # Optional: can skip or leave undefined

        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        pr_auc = average_precision_score(y_test, y_proba)

        # Log metrics
        mlflow.log_metric("Precision", report['1']['precision'])
        mlflow.log_metric("Recall", report['1']['recall'])
        mlflow.log_metric("F1", report['1']['f1-score'])
        mlflow.log_metric("Train_Time_seconds", round(train_time, 2))

        # Log model
        # mlflow.sklearn.log_model(model, model_name)
        
        if isinstance(model, xgb.XGBClassifier):
            mlflow.xgboost.log_model(model, name=model_name, input_example=X_test.iloc[:1])
        elif isinstance(model, lgb.LGBMClassifier):
            mlflow.lightgbm.log_model(model, name=model_name, input_example=X_test.iloc[:1])
        else:
            mlflow.sklearn.log_model(model, name=model_name, input_example=X_test.iloc[:1])

        # Confusion matrix
        fig, ax = plt.subplots()
        ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)
        plt.title("Confusion Matrix")
        path = f"conf_matrix_{model_name}_{sampling_method}.png"
        plt.savefig(path)
        mlflow.log_artifact(path)
        plt.close()



2025/06/22 11:16:07 INFO mlflow.tracking.fluent: Experiment with name 'Loan Default Model Comparison V4' does not exist. Creating a new experiment.


In [28]:

X_train_scaled, X_test_scaled, X_train_unscaled, X_test_unscaled, y_train, y_test, scaler = stratified_split_and_scale(
    user_data, target_col='Default'
)

resample_methods = ['none', 'undersample', 'oversample', 'smote', 'smote_tomek' ]
# resample_methods = ['none', 'undersample']

thresholds = [0.3, 0.5, 0.8]

for method in resample_methods:
    print(f"\n============================")
    print(f"🔄 Sampling: {method.upper()}")
    print("============================")

    # Logistic Regression
    X_res, y_res = apply_resampling(X_train_scaled, y_train, method)
    model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        evaluate_with_mlflow(model, X_test_scaled, y_test, "LogisticRegression", method, X_res, y_res, end - start, threshold)
    # evaluate_with_mlflow(model, X_test_scaled, y_test, "LogisticRegression", method, X_res, y_res, end - start)

    # Decision Tree
    X_res, y_res = apply_resampling(X_train_scaled, y_train, method)
    model = DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        evaluate_with_mlflow(model, X_test_scaled, y_test, "DecisionTree", method, X_res, y_res, end - start, threshold)

    # Random Forest
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        evaluate_with_mlflow(model, X_test_unscaled, y_test, "RandomForest", method, X_res, y_res, end - start, threshold)

    # XGBoost
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = xgb.XGBClassifier(
        n_estimators=100,
        eval_metric='logloss',
        scale_pos_weight=(y_res == 0).sum() / (y_res == 1).sum(),  # handle imbalance
        random_state=42
    )
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        evaluate_with_mlflow(model, X_test_unscaled, y_test, "XGBoost", method, X_res, y_res, end - start, threshold)

    # LightGBM
    X_res, y_res = apply_resampling(X_train_unscaled, y_train, method)
    model = lgb.LGBMClassifier(
        n_estimators=100,
        class_weight='balanced',  # handles imbalance natively
        random_state=42
    )
    start = time.time()
    model.fit(X_res, y_res)
    end = time.time()
    for threshold in thresholds:
        evaluate_with_mlflow(model, X_test_unscaled, y_test, "LightGBM", method, X_res, y_res, end - start, threshold)

    # Balanced Random Forest (only on 'none')
    if method == 'none':
        model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
        start = time.time()
        model.fit(X_train_unscaled, y_train)
        end = time.time()
        for threshold in thresholds:
            evaluate_with_mlflow(model, X_test_unscaled, y_test, "BalancedRandomForest", method, X_train_unscaled, y_train, end - start, threshold)



🔄 Sampling: NONE

🔁 NONE - Before: Counter({0: 89608, 1: 7876})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 NONE - Before: Counter({0: 89608, 1: 7876})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 NONE - Before: Counter({0: 89608, 1: 7876})


...

c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co


🔁 SMOTE_TOMEK - Before: Counter({0: 89608, 1: 7876})
✅ SMOTE_TOMEK - After: Counter({0: 89184, 1: 89184})


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [11:26:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  


🔁 SMOTE_TOMEK - Before: Counter({0: 89608, 1: 7876})
✅ SMOTE_TOMEK - After: Counter({0: 89184, 1: 89184})
[LightGBM] [Info] Number of positive: 89184, number of negative: 89184
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10981
[LightGBM] [Info] Number of data points in the train set: 178368, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Pankaj\loan-data-analysis\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data co

In [29]:
# results_df.sort_values(by=['Model','Sampling'])